In [2]:
# SARIMA example
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
import pandas as pd

In [4]:
# load dataset
file_list=['BTC-all','BTCUSDT-1h','ETH-all','ETHUSDT-1h']


spilt_list=[0.3]

files_list=list()
spilts_list=list()
Classifiers_list=list()

train_mae=list()
train_rmse=list()
test_mae=list()
test_rmse=list()

Expected_revenues=list()
Actual_revenue=list()
Expected_total_revenues=list()

In [5]:
def  Actual_Revenue(actual_prices:list(),predicted_prices:list()):
    predicted_len=(len(predicted_prices) - 1)
    predicted_changes = [predicted_prices[i + 1] - predicted_prices[i] for i in range(predicted_len)]
    expected_revenue=0
    actual_revenue=actual_prices[-1]-actual_prices[0]

    for i in range(len(predicted_prices) - 1):  # Assuming a 7-day test period
        if predicted_changes[i] > 0:  # Buy signal
            expected_revenue += predicted_changes[i]

    return expected_revenue/predicted_len,actual_revenue/predicted_len

def  Expected_Revenue(actual_prices:list(),predicted_prices:list()):
    predicted_len=(len(predicted_prices) - 1)
    predicted_changes = [predicted_prices[i + 1] - predicted_prices[i] for i in range(predicted_len)]
    expected_revenue=0
    for i in range(len(predicted_prices)-1):  # Assuming a 7-day test period
        if predicted_changes[i] > 0:  # Buy signal
            expected_revenue += (actual_prices[i + 1] - actual_prices[i])
        elif predicted_changes[i] < 0:  # Sell signal
            expected_revenue += (actual_prices[i] - actual_prices[i + 1])
    return expected_revenue/predicted_len

In [6]:
for file in file_list:
    for splits in spilt_list:
        for i in range(4):
            for j in range(4):
                #dataset
                # print(file)
                # print(str(splits),end='\n')
                
                path =f'../Data/{file}.csv'
                files_list.append(file)
                spilts_list.append(splits)
                Classifiers_list.append(f'ARIMA({i},1,{j})')

                df = pd.read_csv(path)

                if splits==7:
                    train_spilt=int(len(df)-splits)
                else:
                    train_spilt=len(df)-int(len(df)*splits)
                train_df,test_df=df[:train_spilt],df[train_spilt:]


                data=list(train_df['Close'])
                # fit model
                model = SARIMAX(data, order=(i, 1, j), seasonal_order=(0, 0, 0,0))
                model_fit = model.fit(disp=True)

                # make prediction
                test_predict = model_fit.predict(start=len(data), end=(len(data)+test_df.shape[0]-1))
                train_predict = model_fit.predict(start=0, end=(train_df.shape[0]-1))
                        
                
                # Er,Ar=Actual_Revenue(list(test_df['Close']),list(test_predict)) 
                # Expected_revenues.append(Er)
                # Actual_revenue.append(Ar)
                # Expected_total_revenues.append(Expected_Revenue(list(test_df['Close']),list(test_predict)))
                
                # print('-'*60)
            
                train_mae.append(mean_absolute_error(train_df['Close'], list(train_predict)))
                train_rmse.append(mean_squared_error(train_df['Close'], list(train_predict))**0.5)                
                
                test_mae.append(mean_absolute_error(test_df['Close'], list(test_predict)))
                test_rmse.append(mean_squared_error(test_df['Close'], list(test_predict))**0.5)
                

c:\Users\nptu\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\nptu\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\nptu\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\nptu\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Usin

In [7]:
Summary={'Data':files_list,'Classifier':Classifiers_list,'Test Split':spilts_list,
            'train_mae':train_mae,          
            'train_rmse':train_rmse,          
            'test_mae':test_mae,           
            'test_rmse':test_rmse            
         }

df_Summary = pd.DataFrame(Summary)
df_Summary.to_excel("Summary-ARIMA .xlsx",index=False)  